# Graphviz and Python
*Nodes and Edges and Graphs Oh My*

Not this kind of graph ...

![Image](notThatKindOfGraph.svg)

This kind of graph ...

![Image](example1.svg)

graphviz is package of command line programs that allow diagrams to be specified in code and then saved in a number of different formats (.svg, .png, .jpeg, and many more.)

A typical graphviz command:

`dot -Tsvg example1.dot > example1.svg`

Note: `dot` is one of a number of programs that Graphviz refers to as filters.  Filters use different strategies for positioning elements of the graphs.  DOT is also the language used to define

So what's in a .dot file?
    
```dot
digraph example1{  // directed graph
    rankdir="LR";  // set attribute of graph
    A -> B -> C;   // 3 nodes and the edges between
}
```

![example1](example1.svg)


```dot
graph example2 {
  rankdir="LR";
  label="Title of digraph";

  // define a node w/ attributes
  "node 1" [fillcolor="lightgreen"; style="filled"];

  // define another node w/ attributes
  B [label="node 2"];

  // un-directed edge with attributes
  "node 1" -- B [label="directionality\nis overrated"];
}
```

![example2](example2.svg)

```dot
digraph example3{
  rankdir="LR";
  label="example3";
  subgraph rectangular_nodes {
    node [shape="rectangle"];
    A;
  };
  subgraph cluster_1{
    label="cluster 1";
    B;
  }
  A -> B;
}
```

![example3](example3.svg)

So let's use a Python library to generate the DOT code

![stack1](stack1.svg)

To install the graphviz library.  

`pip install graphviz`

Make sure the `dot` program is on your `PATH`

In [3]:
from graphviz import Digraph

graph = Digraph('graphviz1', comment='Graph Label')
graph.node('A', "Node1")
graph.node('B', "Node2")
graph.node('C', "Node3")
graph.edges(['AB', 'BC'])

In [5]:
print(graph.source)

// Graph Label
digraph graphviz1 {
	A [label=Node1]
	B [label=Node2]
	C [label=Node3]
	A -> B
	B -> C
}

